In [35]:
import datetime
import time
import pandas as pd
from sqlalchemy import create_engine
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.support.wait import WebDriverWait
import selenium.webdriver.support.expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from samoyan_pack.messenger1 import messenger1
import json
pd.set_option('display.max_columns', None)

# selenium 4
import random
import requests
from bs4 import BeautifulSoup

options = Options()
options.add_experimental_option(
    "prefs",
    {
        "download.default_directory": "./",
        "download.prompt_for_download": False,
        "download.directory_upgrade": True,
        "profile.default_content_setting_values.automatic_downloads": True,
    },
)
driver = webdriver.Chrome(options=options)
driver.get("https://www.12306.cn/index/")

In [36]:
def element_waiter(xpath: str):
    WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, xpath)))


def tablize_order_page(driver):
    # Find the table element (adjust the locator as needed)
    table = driver.find_element(By.ID, "t-list")  # Example using XPath
    # Get the HTML content of the table
    table_html = table.get_attribute("outerHTML")
    # Use pandas to read the HTML table
    df = pd.read_html(table_html)[0]  # Extract the first table from the page

    # Extract IDs from table rows (adjust XPath based on your HTML structure)
    ids = []
    rows = table.find_elements(By.CLASS_NAME, "train")
    for row in rows:
        try:
            id_attr = row.get_attribute("id")  # Get the 'id' attribute of the row
            ids.append(id_attr)
        except:
            ids.append(None)  # Handle cases where 'id' attribute is not found

    # Add the 'id' column to the DataFrame
    df["id"] = ids
    return df


def list_to_dataframe(data_list):
    """Converts a list of pipe-delimited strings to a Pandas DataFrame.

    Args:
        data_list: A list of strings, where each string represents a row.

    Returns:
        A Pandas DataFrame, or None if the input is invalid.
    """
    if not data_list:  # Handle empty list case
        return pd.DataFrame()

    try:
        # Split each string into a list of values
        split_data = [row.split("|") for row in data_list]

        # Determine the number of columns (maximum number of splits)
        num_cols = max(len(row) for row in split_data)

        # Pad shorter rows with empty strings to ensure consistent columns
        for row in split_data:
            while len(row) < num_cols:
                row.append("")  # Or you could append None, np.nan, etc.

        # Create the DataFrame
        df = pd.DataFrame(split_data)
        return df
    except AttributeError:  # Handle cases where elements are not string
        print("Error: Input list contains non-string elements.")
        return None
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return None

In [31]:
# deng lu hou -- zai ji xu

 qiang piao !!

True

4
4


In [56]:
while (datetime.datetime.now().hour == 7) & (datetime.datetime.now().minute <= 28):
    print(datetime.datetime.now().minute)
    time.sleep(16)
    try:
        driver.find_element(By.ID, "query_ticket").click()
    except:
        pass
t = 0
# driver.refresh()
while t < 999:
    # click the query button indefinately, to see the ticket is available or not.
    try:
        driver.find_element(By.ID, "query_ticket").click()
    except:
        pass
    time.sleep(random.randint(1, 15) / 10)
    t += 1
    # print(t)
    # this is the ticket I want.
    yuding_button_in_table_ = '//*[@id="ticket_550000Z19607_01_06"]/td[13]/a'
    
    try:
        driver.find_element(By.XPATH, yuding_button_in_table_).click()
        seattype_ = '//*[@id="seatType_1"]'
        element_waiter(seattype_)
        driver.find_element(By.XPATH, seattype_).click()
        try:
            # ruan wo huozhe ying wo
            driver.find_element(By.XPATH, '//*[@id="seatType_1"]/option[3]').click()
        except:
            driver.find_element(By.XPATH, '//*[@id="seatType_1"]/option[1]').click()
        # select passenger
        checkbox_select_passenger_ = '//*[@id="normal_passenger_id"]/li[1]/label'
        element_waiter(checkbox_select_passenger_)
        driver.find_element(By.XPATH, checkbox_select_passenger_).click()
        driver.find_element(By.XPATH, '//*[@id="submitOrder_id"]').click()
        driver.find_element(By.XPATH,'//*[@id="qr_submit_id"]').click()
    except:
        # print("no find")
        pass
    # break

KeyboardInterrupt: 

In [95]:
# driver.find_element(By.XPATH,'//*[@id="qr_submit_id"]').click()

In [39]:
def tablize_order_page(driver):
    # Find the table element (adjust the locator as needed)
    table = driver.find_element(By.ID, "t-list")  # Example using XPath
    # Get the HTML content of the table
    table_html = table.get_attribute("outerHTML")
    # Use pandas to read the HTML table
    df = pd.read_html(table_html)[0]  # Extract the first table from the page

    # Extract IDs from table rows (adjust XPath based on your HTML structure)
    ids = []
    rows = table.find_elements(By.CLASS_NAME, "train")
    for row in rows:
        try:
            id_attr = row.get_attribute("id")  # Get the 'id' attribute of the row
            ids.append(id_attr)
        except:
            ids.append(None)  # Handle cases where 'id' attribute is not found

    # Add the 'id' column to the DataFrame
    df["id"] = ids
    return df

In [55]:
# while True:
df = tablize_order_page(driver)
while (
    (
        df[df["车次"].str.startswith("Z196")][
            [
                "软卧/动卧 一等卧",
                "硬卧 二等卧",
            ]
        ]
        == "候补"
    ).sum(axis=1)
    == 2
).all():
    time.sleep(16)
    try:
        driver.find_element(By.ID, "query_ticket").click()
    except:
        pass
yuding_button_in_table_ = '//*[@id="ticket_550000Z19607_01_06"]/td[13]/a'

try:
    driver.find_element(By.XPATH, yuding_button_in_table_).click()
    seattype_ = '//*[@id="seatType_1"]'
    element_waiter(seattype_)
    driver.find_element(By.XPATH, seattype_).click()
    try:
        # ruan wo huozhe ying wo
        driver.find_element(By.XPATH, '//*[@id="seatType_1"]/option[3]').click()
    except:
        driver.find_element(By.XPATH, '//*[@id="seatType_1"]/option[1]').click()
    # select passenger
    checkbox_select_passenger_ = '//*[@id="normal_passenger_id"]/li[1]/label'
    element_waiter(checkbox_select_passenger_)
    driver.find_element(By.XPATH, checkbox_select_passenger_).click()
    driver.find_element(By.XPATH, '//*[@id="submitOrder_id"]').click()
    driver.find_element(By.XPATH, '//*[@id="qr_submit_id"]').click()
except:
    # print("no find")
    pass

KeyboardInterrupt: 